In [1]:
!pip install SentencePiece transformers datasets evaluate sacrebleu accelerate -U


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
import evaluate

metric = evaluate.load("sacrebleu")

In [4]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

In [5]:
from sklearn.model_selection import train_test_split
import datasets
import pandas as pd

In [6]:
import torch
torch.__version__

'2.1.2+cpu'

In [7]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

e:\Computer_Science\internshipp\Nitesh Kumar\venv\Lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [8]:
from datasets import load_dataset

dataset1 = load_dataset("allenai/wmt22_african", "xho-zul")

In [9]:
def add_id(data, idx):
    # Incrementing idx by 1 to start from 1 instead of 0
    data['id'] = idx + 1
    return data

columns_to_remove = ['laser_score', 'source_sentence_lid', 'target_sentence_lid']
dataset1 = dataset1.remove_columns(columns_to_remove)

# Map this function over the entire dataset to add 'id' column
dataset_with_id = dataset1.map(add_id, with_indices=True)

def preprocess_function(data):
    # Replace 'source_language_code' and 'target_language_code' with appropriate language codes
    source_lang_token = tokenizer.get_lang_token('zu')
    target_lang_token = tokenizer.get_lang_token('xh')

    # inputs = [source_lang_token + data['translation']['zul']]
    # targets = [target_lang_token + data['translation']['xho']]

    inputs = source_lang_token + data["translation"]['zul']
    targets = target_lang_token + data["translation"]['xho']

    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding='max_length')
    labels = tokenizer(targets, max_length=128, truncation=True, padding='max_length')

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Split the dataset into train and test sets
train_test_split = dataset_with_id["train"].train_test_split(test_size=0.2)
train_dataset = train_test_split["train"]
test_dataset = train_test_split["test"]

I am running with smaller dataset , so you can see its running. :)

In [10]:
# # # Try running the map function with a smaller subset to debug
# train_small_subset = train_dataset.select(range(10))
# test_small_subset = test_dataset.select(range(10))


# tokenized_subset_train = train_small_subset.map(preprocess_function, batched=False)
# tokenized_subset_test = test_small_subset.map(preprocess_function, batched=False)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Actual Dataset is Below, I will uncomment it.

In [11]:
tokenized_train = train_dataset.map(preprocess_function, batched=False)
tokenized_test = test_dataset.map(preprocess_function, batched=False)

In [12]:
# !wget https://dl.fbaipublicfiles.com/m2m_100/418M_last_checkpoint.pt

In [13]:
# !wget https://dl.fbaipublicfiles.com/m2m_100/spm.128k.model

In [14]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import load_dataset

# Define your custom dataset and load it using the datasets library
# custom_dataset = load_dataset("path/to/your/custom/dataset")

# Initialize the M2M-100 tokenizer and model
model_name = "facebook/m2m100_418M"
tokenizer = M2M100Tokenizer.from_pretrained(model_name)
model = M2M100ForConditionalGeneration.from_pretrained(model_name)


In [15]:
# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="test_1",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=1000,
    num_train_epochs=2,
    logging_dir="./logs",
    push_to_hub=True,
)

# Initialize data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
)

# Initialize the Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_train, #tokenized_subset_train,
    eval_dataset=tokenized_test, #tokenized_subset_test,
)

In [16]:
# Train the model
trainer.train()

  0%|          | 0/4 [00:00<?, ?it/s]

{'train_runtime': 95.9611, 'train_samples_per_second': 0.208, 'train_steps_per_second': 0.042, 'train_loss': 10.129739761352539, 'epoch': 2.0}


TrainOutput(global_step=4, training_loss=10.129739761352539, metrics={'train_runtime': 95.9611, 'train_samples_per_second': 0.208, 'train_steps_per_second': 0.042, 'train_loss': 10.129739761352539, 'epoch': 2.0})

In [17]:

# Evaluate the model on the test dataset
results = trainer.evaluate(eval_dataset=tokenized_test)#tokenized_subset_test)
results
# Save the trained model
trainer.save_model("./custom_m2m100_model")

# You can use the saved model for translation or further fine-tuning


  0%|          | 0/2 [00:00<?, ?it/s]

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}


model.safetensors:   0%|          | 0.00/1.94G [00:00<?, ?B/s]